### Final Project 

In [1]:
import numpy as np
import pandas as pd

%load_ext lab_black

#### Import covid-19 data

In [2]:
src = pd.read_csv(
    "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv",
    dtype={"fips": str},
)

In [3]:
len(src)

2011607

#### Only look at California

In [4]:
df = src[src["state"] == "California"].copy()

In [5]:
len(df)

36934

In [6]:
df.dtypes

date       object
county     object
state      object
fips       object
cases       int64
deaths    float64
dtype: object

In [7]:
df["deaths"] = df["deaths"].astype(int)

In [8]:
df.dtypes

date      object
county    object
state     object
fips      object
cases      int64
deaths     int64
dtype: object

In [9]:
df.tail()

,date,county,state,fips,cases,deaths
2008596,2021-12-13,Tulare,California,06107,87731,1115
2008597,2021-12-13,Tuolumne,California,06109,8015,148
2008598,2021-12-13,Ventura,California,06111,105705,1197
2008599,2021-12-13,Yolo,California,06113,21991,267
2008600,2021-12-13,Yuba,California,06115,11421,92


In [10]:
df.head()

,date,county,state,fips,cases,deaths
5,2020-01-25,Orange,California,06059,1,0
9,2020-01-26,Los Angeles,California,06037,1,0
10,2020-01-26,Orange,California,06059,1,0
14,2020-01-27,Los Angeles,California,06037,1,0
15,2020-01-27,Orange,California,06059,1,0


#### Deal with 4 digit fips

In [11]:
df["fips"] = df["fips"].str.zfill(5)

#### Only look at the latest date

In [12]:
df["nudate"] = pd.to_datetime(df["date"])

In [13]:
covid_recent = df[df["date"] == df["date"].max()]
covid_recent.head()

,date,county,state,fips,cases,deaths,nudate
2008543,2021-12-13,Alameda,California,06001,127265,1518,2021-12-13
2008544,2021-12-13,Alpine,California,06003,109,0,2021-12-13
2008545,2021-12-13,Amador,California,06005,5868,69,2021-12-13
2008546,2021-12-13,Butte,California,06007,25885,326,2021-12-13
2008547,2021-12-13,Calaveras,California,06009,4571,89,2021-12-13


In [14]:
covid_recent.describe().round()

,cases,deaths
count,58.0,58.0
mean,89200.0,1303.0
std,218779.0,3754.0
min,109.0,0.0
25%,7338.0,82.0
50%,24342.0,291.0
75%,80319.0,995.0
max,1548861.0,27341.0


In [15]:
covid_recent[["county", "cases", "deaths"]].head(10)

,county,cases,deaths
2008543,Alameda,127265,1518
2008544,Alpine,109,0
2008545,Amador,5868,69
2008546,Butte,25885,326
2008547,Calaveras,4571,89
2008548,Colusa,3322,21
2008549,Contra Costa,105432,1065
2008550,Del Norte,3919,42
2008551,El Dorado,18730,170
2008552,Fresno,159332,2310


#### Which county has the most cases 

In [16]:
covid_recent.groupby(["county"]).agg(
    {"cases": "mean", "deaths": "sum"}
).reset_index().sort_values("cases", ascending=False).head(15)

,county,cases,deaths
18,Los Angeles,1548861,27341
36,San Diego,416102,4378
32,Riverside,395013,5502
35,San Bernardino,382217,6011
29,Orange,339106,5825
33,Sacramento,171597,2483
14,Kern,160199,1875
9,Fresno,159332,2310
42,Santa Clara,155142,1946
0,Alameda,127265,1518


In [57]:
covid_recent.groupby(["county"]).agg(
    {"cases": "mean", "deaths": "mean"}
).reset_index().sort_values("deaths", ascending=False).head(15)

,county,cases,deaths
18,Los Angeles,1548861,27341
35,San Bernardino,382217,6011
29,Orange,339106,5825
32,Riverside,395013,5502
36,San Diego,416102,4378
33,Sacramento,171597,2483
9,Fresno,159332,2310
42,Santa Clara,155142,1946
14,Kern,160199,1875
38,San Joaquin,109268,1846


#### Import some demographic data and merge them

In [17]:
data = pd.read_csv("./acs5_2019_race_counties.csv")

In [18]:
age = pd.read_csv("./counties_ca_median_age (1).csv")

In [19]:
demographic = pd.merge(data, age, left_on="name", right_on="county", how="left",)

In [20]:
demographic.drop(["name"], axis=1, inplace=True)

In [21]:
len(demographic)

58

In [22]:
demographic.dtypes

geoid                int64
universe             int64
white_alone          int64
black_alone          int64
latino_alone         int64
asian_alone          int64
other_multirace      int64
id                  object
county              object
population           int64
median_age         float64
fips                 int64
dtype: object

In [23]:
demographic["county"] = demographic["county"].str.replace(" County", "", regex=False)

In [24]:
demographic.head()

,geoid,universe,white_alone,black_alone,latino_alone,asian_alone,other_multirace,id,county,population,median_age,fips
0,6001,1656754,520447,171168,371019,509283,84837,0500000US06001,Alameda,1656754,37.6,1
1,6005,38429,30035,848,5340,529,1677,0500000US06005,Amador,38429,50.5,5
2,6007,225817,162537,3342,36916,10533,12489,0500000US06007,Butte,225817,37.1,7
3,6009,45514,36837,315,5489,703,2170,0500000US06009,Calaveras,45514,52.0,9
4,6011,21454,7576,256,12738,311,573,0500000US06011,Colusa,21454,35.3,11


#### Merge the demographic data with latest covid data

In [25]:
covid_new = pd.merge(
    covid_recent, demographic, left_on="county", right_on="county", how="left",
)

In [26]:
len(covid_new)

58

In [27]:
covid_new.head()

,date,county,state,fips_x,cases,deaths,nudate,geoid,universe,white_alone,black_alone,latino_alone,asian_alone,other_multirace,id,population,median_age,fips_y
0,2021-12-13,Alameda,California,06001,127265,1518,2021-12-13,6001,1656754,520447,171168,371019,509283,84837,0500000US06001,1656754,37.6,1
1,2021-12-13,Alpine,California,06003,109,0,2021-12-13,6003,1039,551,9,130,10,339,0500000US06003,1039,52.2,3
2,2021-12-13,Amador,California,06005,5868,69,2021-12-13,6005,38429,30035,848,5340,529,1677,0500000US06005,38429,50.5,5
3,2021-12-13,Butte,California,06007,25885,326,2021-12-13,6007,225817,162537,3342,36916,10533,12489,0500000US06007,225817,37.1,7
4,2021-12-13,Calaveras,California,06009,4571,89,2021-12-13,6009,45514,36837,315,5489,703,2170,0500000US06009,45514,52.0,9


In [28]:
covid_new.tail()

,date,county,state,fips_x,cases,deaths,nudate,geoid,universe,white_alone,black_alone,latino_alone,asian_alone,other_multirace,id,population,median_age,fips_y
53,2021-12-13,Tulare,California,06107,87731,1115,2021-12-13,6107,461898,131716,5785,298245,15573,10579,0500000US06107,461898,31.0,107
54,2021-12-13,Tuolumne,California,06109,8015,148,2021-12-13,6109,54045,43341,948,6617,855,2284,0500000US06109,54045,48.4,109
55,2021-12-13,Ventura,California,06111,105705,1197,2021-12-13,6111,847263,384818,14297,361601,62131,24416,0500000US06111,847263,38.1,111
56,2021-12-13,Yolo,California,06113,21991,267,2021-12-13,6113,217352,101452,5279,68704,31038,10879,0500000US06113,217352,31.0,113
57,2021-12-13,Yuba,California,06115,11421,92,2021-12-13,6115,76360,42145,2554,21563,5371,4727,0500000US06115,76360,32.8,115


#### There are no strong relationship between race populations and covid cases/deaths

In [29]:
covid_new["white_percent"] = covid_new["white_alone"] / covid_new["population"]
covid_new["black_percent"] = covid_new["black_alone"] / covid_new["population"]
covid_new["hispanic_percent"] = covid_new["latino_alone"] / covid_new["population"]
covid_new["asian_percent"] = covid_new["asian_alone"] / covid_new["population"]

In [33]:
corr = covid_new[
    [
        "deaths",
        "white_percent",
        "black_percent",
        "hispanic_percent",
        "asian_percent",
        "cases",
    ]
].corr(method="pearson")

In [34]:
corr

,deaths,white_percent,black_percent,hispanic_percent,asian_percent,cases
deaths,1.000000,-0.343475,0.354417,0.244218,0.233124,0.992898
white_percent,-0.343475,1.000000,-0.484299,-0.848792,-0.470596,-0.375199
black_percent,0.354417,-0.484299,1.000000,0.174925,0.464172,0.391906
hispanic_percent,0.244218,-0.848792,0.174925,1.000000,-0.002795,0.260426
asian_percent,0.233124,-0.470596,0.464172,-0.002795,1.000000,0.267266
cases,0.992898,-0.375199,0.391906,0.260426,0.267266,1.000000


#### Also no strong relationship between median age and cases/deaths 

In [35]:
covid_new[["median_age", "cases", "deaths"]].corr(method="pearson")

,median_age,cases,deaths
median_age,1.000000,-0.234253,-0.201877
cases,-0.234253,1.000000,0.992898
deaths,-0.201877,0.992898,1.000000


In [37]:
covid_new.groupby(["county"]).agg(
    {"cases": "mean", "deaths": "sum", "population": "mean"}
).reset_index().sort_values("cases", ascending=False).head(15)

,county,cases,deaths,population
18,Los Angeles,1548861,27341,10081570
36,San Diego,416102,4378,3316073
32,Riverside,395013,5502,2411439
35,San Bernardino,382217,6011,2149031
29,Orange,339106,5825,3168044
33,Sacramento,171597,2483,1524553
14,Kern,160199,1875,887641
9,Fresno,159332,2310,984521
42,Santa Clara,155142,1946,1927470
0,Alameda,127265,1518,1656754


#### The similar findings shown in the April covid data

In [39]:
covid_april = df[df["date"] == "2021-04-20"]
covid_april.head(10)

,date,county,state,fips,cases,deaths,nudate
1238765,2021-04-20,Alameda,California,06001,85604,1477,2021-04-20
1238766,2021-04-20,Alpine,California,06003,88,0,2021-04-20
1238767,2021-04-20,Amador,California,06005,3628,46,2021-04-20
1238768,2021-04-20,Butte,California,06007,11601,196,2021-04-20
1238769,2021-04-20,Calaveras,California,06009,2050,51,2021-04-20
1238770,2021-04-20,Colusa,California,06011,2204,16,2021-04-20
1238771,2021-04-20,Contra Costa,California,06013,67157,782,2021-04-20
1238772,2021-04-20,Del Norte,California,06015,1290,7,2021-04-20
1238773,2021-04-20,El Dorado,California,06017,9726,109,2021-04-20
1238774,2021-04-20,Fresno,California,06019,100702,1646,2021-04-20


In [41]:
covid_early = pd.merge(
    covid_april, demographic, left_on="county", right_on="county", how="left",
)
covid_early.head()

,date,county,state,fips_x,cases,deaths,nudate,geoid,universe,white_alone,black_alone,latino_alone,asian_alone,other_multirace,id,population,median_age,fips_y
0,2021-04-20,Alameda,California,06001,85604,1477,2021-04-20,6001,1656754,520447,171168,371019,509283,84837,0500000US06001,1656754,37.6,1
1,2021-04-20,Alpine,California,06003,88,0,2021-04-20,6003,1039,551,9,130,10,339,0500000US06003,1039,52.2,3
2,2021-04-20,Amador,California,06005,3628,46,2021-04-20,6005,38429,30035,848,5340,529,1677,0500000US06005,38429,50.5,5
3,2021-04-20,Butte,California,06007,11601,196,2021-04-20,6007,225817,162537,3342,36916,10533,12489,0500000US06007,225817,37.1,7
4,2021-04-20,Calaveras,California,06009,2050,51,2021-04-20,6009,45514,36837,315,5489,703,2170,0500000US06009,45514,52.0,9


In [42]:
covid_early["white_percent"] = covid_early["white_alone"] / covid_early["population"]
covid_early["black_percent"] = covid_early["black_alone"] / covid_early["population"]
covid_early["hispanic_percent"] = (
    covid_early["latino_alone"] / covid_early["population"]
)
covid_early["asian_percent"] = covid_early["asian_alone"] / covid_early["population"]

In [43]:
corr_early = covid_early[
    [
        "deaths",
        "white_percent",
        "black_percent",
        "hispanic_percent",
        "asian_percent",
        "cases",
    ]
].corr(method="pearson")

In [45]:
corr_early

,deaths,white_percent,black_percent,hispanic_percent,asian_percent,cases
deaths,1.000000,-0.335600,0.340930,0.233986,0.238679,0.995689
white_percent,-0.335600,1.000000,-0.484299,-0.848792,-0.470596,-0.362022
black_percent,0.340930,-0.484299,1.000000,0.174925,0.464172,0.372468
hispanic_percent,0.233986,-0.848792,0.174925,1.000000,-0.002795,0.253848
asian_percent,0.238679,-0.470596,0.464172,-0.002795,1.000000,0.254103
cases,0.995689,-0.362022,0.372468,0.253848,0.254103,1.000000


In [46]:
covid_early[["median_age", "cases", "deaths"]].corr(method="pearson")

,median_age,cases,deaths
median_age,1.000000,-0.216682,-0.189088
cases,-0.216682,1.000000,0.995689
deaths,-0.189088,0.995689,1.000000


#### Maybe need a case intensity percentage

In [48]:
covid_new["case_percent"] = (covid_new["cases"] / covid_new["population"]) * 100

#### Which county has the most cases intensity

In [51]:
covid_new.groupby(["county"]).agg(
    {"case_percent": "mean", "deaths": "sum", "population": "mean"}
).reset_index().sort_values("case_percent", ascending=False).head(15)

,county,case_percent,deaths,population
17,Lassen,26.052956,61,30818
15,Kings,23.799696,372,150691
12,Imperial,21.999325,786,180701
53,Tulare,18.993587,1115,461898
14,Kern,18.047724,1875,887641
35,San Bernardino,17.785551,6011,2149031
19,Madera,17.204841,315,155433
49,Stanislaus,17.115984,1452,543194
23,Merced,17.028027,685,271382
10,Glenn,16.535602,51,27976


In [52]:
covid_new[
    [
        "white_percent",
        "black_percent",
        "hispanic_percent",
        "asian_percent",
        "case_percent",
    ]
].corr(method="pearson")

,white_percent,black_percent,hispanic_percent,asian_percent,case_percent
white_percent,1.000000,-0.484299,-0.848792,-0.470596,-0.377062
black_percent,-0.484299,1.000000,0.174925,0.464172,0.205291
hispanic_percent,-0.848792,0.174925,1.000000,-0.002795,0.583787
asian_percent,-0.470596,0.464172,-0.002795,1.000000,-0.335007
case_percent,-0.377062,0.205291,0.583787,-0.335007,1.000000


#### Showing strong relationship 

In [53]:
# !pip install altair
import altair as alt

alt.Chart(
    covid_new[
        [
            "white_percent",
            "black_percent",
            "hispanic_percent",
            "asian_percent",
            "case_percent",
        ]
    ]
).mark_circle(size=60).encode(x="hispanic_percent", y="case_percent",).interactive()

alt.Chart(...)

In [54]:
covid_new[["median_age", "case_percent"]].corr(method="pearson")

,median_age,case_percent
median_age,1.000000,-0.486266
case_percent,-0.486266,1.000000


In [55]:
alt.Chart(covid_new[["median_age", "case_percent",]]).mark_circle(size=60).encode(
    x="median_age", y="case_percent",
).interactive()

alt.Chart(...)